# 5.7.2 오차역전파법을 적용한 신경망 구현하기

In [24]:
import sys, os
print(os.pardir)
sys.path.append(os.pardir)
import numpy as np
from deep_learning_github.common.layers import *
from deep_learning_github.common.gradient import numerical_gradient
from collections import OrderedDict

..


In [25]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
            
        return x
    
    # x :입력 데이터, t : 정답 레이블
    def loss(self, x, y):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
            
        accuracy = np.sum(y == t) / float(x.shape[0])
        
        return accuracy
    
    # x :입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W : self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)
        
        # 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads        

# 5.7.3 오차역전파법으로 구한 기울기 검증하기

In [26]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from deep_learning_github.dataset.mnist import load_mnist
from deep_learning_github.ch05.two_layer_net import TwoLayerNet

#### 데이터 읽기

In [27]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
x_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [30]:
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

#### 각 가중치의 차이의 절댓값을 구한 후, 그 절대값들의 평균을 낸다.

In [31]:
for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":" + str(diff))

W1:3.5260077801482067e-10
b1:2.0524325647613537e-09
W2:4.5597265679963685e-09
b2:1.3991338485147554e-07


In [32]:
x_train.shape

(60000, 784)

# 오차역전파법을 사용한 학습 구현하기

In [37]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from deep_learning_github.dataset.mnist import load_mnist
from deep_learning_github.ch05.two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 오차역전파법으로 기울기를 구한다.
    grad = network.gradient(x_batch, t_batch)
    
    # 갱신
    for key in ('W1','b1','W2','b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train_acc : " + str(train_acc), "test_acc : " + str(test_acc))

train_acc : 0.13061666666666666 test_acc : 0.1221
train_acc : 0.9073833333333333 test_acc : 0.9115
train_acc : 0.9228833333333334 test_acc : 0.9252
train_acc : 0.9338833333333333 test_acc : 0.9333
train_acc : 0.9444666666666667 test_acc : 0.9415
train_acc : 0.9508333333333333 test_acc : 0.9479
train_acc : 0.9556666666666667 test_acc : 0.9529
train_acc : 0.9572333333333334 test_acc : 0.9533
train_acc : 0.9610166666666666 test_acc : 0.9566
train_acc : 0.9669 test_acc : 0.9608
train_acc : 0.9681 test_acc : 0.9623
train_acc : 0.9709833333333333 test_acc : 0.9636
train_acc : 0.9721333333333333 test_acc : 0.9652
train_acc : 0.9744 test_acc : 0.9668
train_acc : 0.9749666666666666 test_acc : 0.9666
train_acc : 0.9772666666666666 test_acc : 0.9687
train_acc : 0.9786833333333333 test_acc : 0.9697
